In [72]:
import pandas as pd
import numpy as np

In [73]:
# load in (cleaned) table containing race info
df = pd.read_csv(r"C:\Users\gratt\Desktop\emsData\processedCSVs\small\pcrpatientracegroup.csv")

In [74]:
df.head()

,PcrPatientRaceGroupKey,PcrKey,ePatient_14
0,2088217,1472192,7701003
1,16835623,13273412,2514011
2,18418102,14548133,7701003
3,21377926,17184298,2514011
4,38817731,21699657,2514011


In [75]:
# print list of unique values in race column
print(list(df.ePatient_14.unique()))

[7701003, 2514011, 2514001, 2514005, 2514007, 7701001, 2514009, 2514003]


- 2514001 - American Indian or Alaska Native 
- 2514003 - Asian
- 2514005 - Black or African American 
- 2514007 - Hispanic or Latino
- 2514009 - Native Hawaiian or Other Pacific Islander
- 2514011 - White
- 7701001 - Not Applicable
- 7701003 - Not Recorded

In [76]:
print(f"Number of rows = {len(df.index)}")
print(f"Number of Unique PCRs = {df.PcrKey.nunique()}")

Number of rows = 49555288
Number of Unique PCRs = 48982990


In [79]:
# look at subset of df where one PCR is associated with multiple races
dup = df[df.duplicated('PcrKey',False)]
dup

,PcrPatientRaceGroupKey,PcrKey,ePatient_14
68,158136243,118170967,2514005
69,158136244,118170967,2514007
243,158136383,118171544,2514009
244,158136384,118171544,2514011
570,186751735,118172608,2514011
...,...,...,...
49555146,239103224,182521800,2514007
49555245,239123679,182538440,2514011
49555246,239123678,182538440,2514005
49555265,239124653,182539411,2514011


- For example, this first person, PcrKey = 118170967, is listed as both 2514005 (black) AND 2514007 (hispanic/latino)
- This makes perfect sense

In [80]:
print(f"Number of multi-racial patients = {dup.PcrKey.nunique()}")
print(f"Average races of multi-racial patient = {len(dup)/dup.PcrKey.nunique()}")

Number of multi-racial patients = 492196
Average races of multi-racial patient = 2.1627441100699722


In [81]:
# check race proportions
prop = df['ePatient_14'].value_counts(normalize=True)
prop

2514011    0.432621
7701003    0.301635
2514005    0.156769
2514007    0.065644
7701001    0.023590
2514003    0.010136
2514001    0.006942
2514009    0.002662
Name: ePatient_14, dtype: float64

- 43.3% White
- 30.2% Not Recorded
- 15.7% Black
- 6.6% Hispanic
- 2.4% Not Applicable
- 1.0% Asian
- 0.7% Native American
- 0.2% American Indian or Alaska Native

- I'm gonna drop all of the entries with missing race data
    - Can't do any stats with missing data
- 30% not recorded, 2.3% not applicable
    - This is sorta interesting, not recorded is obviously just missing data but not applicable could be people that don't fit into any of these categories?
    - For now I'll just throw them all out, perhaps will come back if I see fit


In [82]:
# drop not recorded and not applicable
# pandas uses | as the symbol for "or"
nan_obs = df[ (df['ePatient_14'] == 7701003) | (df['ePatient_14'] == 7701001)].index
df.drop(nan_obs, inplace = True)
df.head()

,PcrPatientRaceGroupKey,PcrKey,ePatient_14
1,16835623,13273412,2514011
3,21377926,17184298,2514011
4,38817731,21699657,2514011
6,97354357,72057010,2514011
11,111683931,83733589,2514001


In [83]:
print(len(df.index))
print(49555288 - len(df.index))

33438658
16116630


In [84]:
print(df.PcrKey.nunique())

32866360


- We just threw out 16,116,630 rows
- This should be fine
- Now our total sample size is 33,438,658

In [85]:
# check new proportions
prop = df['ePatient_14'].value_counts(normalize=True)
prop

2514011    0.641134
2514005    0.232328
2514007    0.097283
2514003    0.015021
2514001    0.010288
2514009    0.003946
Name: ePatient_14, dtype: float64

- 64.1% white
- 23.2% black
- 9.7% hispanic / latino
- 1.5% Asian
- 1.0% American Indian or Alaska Native
- 0.4% Native Hawaiian or Other Pacific Islander

In [86]:
dup = df[df.duplicated('PcrKey',False)]
print(len(dup))
print(dup.PcrKey.nunique())

1064494
492196


- Perfect, still 492,196 that are two or more races

In [89]:
df

,PcrPatientRaceGroupKey,PcrKey,ePatient_14
1,16835623,13273412,2514011
3,21377926,17184298,2514011
4,38817731,21699657,2514011
6,97354357,72057010,2514011
11,111683931,83733589,2514001
...,...,...,...
49555278,239125533,182540290,2514005
49555279,239125620,182540377,2514005
49555280,239125626,182540383,2514005
49555282,239127518,182541657,2514011


In [90]:
# create new columns for each race

df['white'] = np.where(df['ePatient_14'] == 2514011, 1, 0)
df['black'] = np.where(df['ePatient_14'] == 2514005, 1, 0)
df['hispanic_latino'] = np.where(df['ePatient_14'] == 2514007, 1, 0)
df['asian'] = np.where(df['ePatient_14'] == 2514003, 1, 0)
df['americanIndian_alaskaNative'] = np.where(df['ePatient_14'] == 2514001, 1, 0)
df['nativeHawaiian_otherPacificIslander'] = np.where(df['ePatient_14'] == 2514009, 1, 0)

In [91]:
df

,PcrPatientRaceGroupKey,PcrKey,ePatient_14,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander
1,16835623,13273412,2514011,1,0,0,0,0,0
3,21377926,17184298,2514011,1,0,0,0,0,0
4,38817731,21699657,2514011,1,0,0,0,0,0
6,97354357,72057010,2514011,1,0,0,0,0,0
11,111683931,83733589,2514001,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
49555278,239125533,182540290,2514005,0,1,0,0,0,0
49555279,239125620,182540377,2514005,0,1,0,0,0,0
49555280,239125626,182540383,2514005,0,1,0,0,0,0
49555282,239127518,182541657,2514011,1,0,0,0,0,0


In [92]:
# drop the now redundant race column and unnecessary key
df = df.drop(columns=['PcrPatientRaceGroupKey', 'ePatient_14'])

In [93]:
df

,PcrKey,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander
1,13273412,1,0,0,0,0,0
3,17184298,1,0,0,0,0,0
4,21699657,1,0,0,0,0,0
6,72057010,1,0,0,0,0,0
11,83733589,0,0,0,0,1,0
...,...,...,...,...,...,...,...
49555278,182540290,0,1,0,0,0,0
49555279,182540377,0,1,0,0,0,0
49555280,182540383,0,1,0,0,0,0
49555282,182541657,1,0,0,0,0,0


In [94]:
df['white'].value_counts(normalize=True)

1    0.641134
0    0.358866
Name: white, dtype: float64

In [95]:
print(len(df.index))
print(df.PcrKey.nunique())

33438658
32866360


In [96]:
df = df.astype(int)
df

,PcrKey,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander
1,13273412,1,0,0,0,0,0
3,17184298,1,0,0,0,0,0
4,21699657,1,0,0,0,0,0
6,72057010,1,0,0,0,0,0
11,83733589,0,0,0,0,1,0
...,...,...,...,...,...,...,...
49555278,182540290,0,1,0,0,0,0
49555279,182540377,0,1,0,0,0,0
49555280,182540383,0,1,0,0,0,0
49555282,182541657,1,0,0,0,0,0


In [97]:
df.dtypes

PcrKey                                 int32
white                                  int32
black                                  int32
hispanic_latino                        int32
asian                                  int32
americanIndian_alaskaNative            int32
nativeHawaiian_otherPacificIslander    int32
dtype: object

In [98]:
dup = df[df.duplicated(keep = False)]

In [100]:
dup

,PcrKey,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander
570,118172608,1,0,0,0,0,0
571,118172608,1,0,0,0,0,0
1726,118224570,1,0,0,0,0,0
1727,118224570,1,0,0,0,0,0
4959,118276662,1,0,0,0,0,0
...,...,...,...,...,...,...,...
18589232,141279151,1,0,0,0,0,0
18589236,141279155,1,0,0,0,0,0
18589237,141279155,1,0,0,0,0,0
18589238,141279156,1,0,0,0,0,0


In [101]:
df = df.drop_duplicates()

In [102]:
df

,PcrKey,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander
1,13273412,1,0,0,0,0,0
3,17184298,1,0,0,0,0,0
4,21699657,1,0,0,0,0,0
6,72057010,1,0,0,0,0,0
11,83733589,0,0,0,0,1,0
...,...,...,...,...,...,...,...
49555278,182540290,0,1,0,0,0,0
49555279,182540377,0,1,0,0,0,0
49555280,182540383,0,1,0,0,0,0
49555282,182541657,1,0,0,0,0,0


In [103]:
test = df[df['PcrKey'] == 118170967]

In [104]:
test

,PcrKey,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander
68,118170967,0,1,0,0,0,0
69,118170967,0,0,1,0,0,0


In [105]:
# group the rows with same PcrKey (multiracial so they have multiple rows)
# then sum
df2 = df.groupby(['PcrKey']).sum()

In [106]:
df2

,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander
PcrKey,,,,,,
13273412,1,0,0,0,0,0
17184298,1,0,0,0,0,0
21699657,1,0,0,0,0,0
72057010,1,0,0,0,0,0
83733589,0,0,0,0,1,0
...,...,...,...,...,...,...
182540290,0,1,0,0,0,0
182540377,0,1,0,0,0,0
182540383,0,1,0,0,0,0


In [107]:
print(len(df2.index))

32866360
33234120


- Started with 49,555,288 rows
- Threw out 16,116,630 rows with missing data
    - Leaving me with 33,438,658 rows
- Then we merged the non-unique PCR rows and left with a CORRECT total number of unique PCRs = 32,866,360

In [108]:
df2 = df2.astype(int)

In [109]:
df2.head(50)

,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander
PcrKey,,,,,,
13273412,1,0,0,0,0,0
17184298,1,0,0,0,0,0
21699657,1,0,0,0,0,0
72057010,1,0,0,0,0,0
83733589,0,0,0,0,1,0
88178354,1,0,0,0,0,0
97455081,1,0,0,0,0,0
102488422,1,0,0,0,0,0
107295362,1,0,0,0,0,0


In [110]:
df2['black'].value_counts(normalize=True)

0    0.764785
1    0.235215
Name: black, dtype: float64

In [111]:
df['white'].value_counts(normalize=True)

1    0.640603
0    0.359397
Name: white, dtype: float64

In [112]:
df2

,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander
PcrKey,,,,,,
13273412,1,0,0,0,0,0
17184298,1,0,0,0,0,0
21699657,1,0,0,0,0,0
72057010,1,0,0,0,0,0
83733589,0,0,0,0,1,0
...,...,...,...,...,...,...
182540290,0,1,0,0,0,0
182540377,0,1,0,0,0,0
182540383,0,1,0,0,0,0


In [113]:
bad = df2[ df2['hispanic_latino'] == 2 ]
bad

,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander
PcrKey,,,,,,


In [114]:
# thank god we got it
df2.to_csv(r"C:\Users\gratt\Desktop\emsData\race.csv")

In [ ]:
# sql stuff which is now irrelevant

In [ ]:
# get SQL schema to create table
schema = pd.io.sql.get_schema(df, 'raceTable'
schema = schema.replace('"','')
schema

In [ ]:
# get cols list for insert statement
cols = list(df.columns)
print(',\n'.join(map(str, cols)))

In [ ]:
# number of variables for insert statement
len(cols)
q = [' ?'] * 7
print(','.join(map(str, q)))

In [9]:
df.to_csv('/Users/patrickburke/Library/CloudStorage/OneDrive-EmoryUniversity/ECON496RW/csvToSQL/race.csv', index=False)